In [ ]:
import os
import torch
from openai import OpenAI
client = OpenAI()

In [23]:
csv_file_path = os.path.join("data", "dataset", "processed", "clean_data.csv")
jsonl_file_path = os.path.join("data", "dataset", "processed", "clean_data.jsonl")
mini_data_path = os.path.join("..", "data", "dataset", "processed", "mini_data.jsonl")
mini_test_path = os.path.join("..", "data", "dataset", "processed", "mini_test.jsonl")

In [24]:
train_file = client.files.create(
  file=open(mini_data_path, "rb"),
  purpose="fine-tune"
)

test_file = client.files.create(
  file=open(mini_test_path, "rb"),
  purpose="fine-tune"
)

In [27]:
#Finetuning on mini dataset
job = client.fine_tuning.jobs.create(
  training_file=train_file.id,
  validation_file=test_file.id,
  model="gpt-4o-mini-2024-07-18",
    hyperparameters={
    "n_epochs":3
  }
)

In [35]:
status = client.fine_tuning.jobs.retrieve(job.id)
print(status)

FineTuningJob(id='ftjob-9KecEmK3QUjy5xFSs6jSfkml', created_at=1731464514, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::ASxOTfhV', finished_at=1731464855, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-ddAZbaTdTMt3bBOAwSkg0NcQ', result_files=['file-HYVxPFOGTxQoY30OblsJQDFn'], seed=1852577086, status='succeeded', trained_tokens=24294, training_file='file-KUKnKVWIr2UvKMxNqnWmKCns', validation_file='file-ms2LnPObPRrQ6YHAVGL00Qkh', estimated_finish=None, integrations=[], user_provided_suffix=None)


In [33]:
fine_tune_results = client.fine_tuning.jobs.retrieve(job.id).result_files
result_file = client.files.retrieve(fine_tune_results[0])
content = client.files.content(result_file.id)
import base64

base64.b64decode(content.text.encode("utf-8"))

with open('band_score_result.csv', "wb") as f:
    f.write(base64.b64decode(content.text.encode("utf-8")))



In [ ]:
completion = client.chat.completions.create(
  model=status.fine_tuned_model,
  messages=[
    {"role": "system", "content": "You are a IELTS writing part examiner who is responsible to provide a band score given an essay"},
    {"role": "user", "content": ""}
  ]
)